# <center>Proactive Server Resource Management</center>

<b><p style="text-align:right;">Kumar Anurag</p></b>
<p style="text-align:right;color:blue">2021FC04016</p>

### Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import time

### Loading the Dataset

In [2]:
dataset = pd.read_csv('dataset_large.csv')
dataset.head()

,Type,Owned By Team,Customer Name,ID,Priority,Created Date Time,Status,Short Description,Category,Service,Subcategory,SLA Resolve By Deadline,Action Taken
0,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2798425,3,6/19/23 2:48,Closed,Host lbjesx02.lbj.is.smartgurucool.com is Down,smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",6/21/23 2:50,"Check if there's any planned outage going on, ..."
1,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2798418,3,6/19/23 2:45,Closed,Host lbjavscan1.lbj.is.smartgurucool.com is Down,smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",6/21/23 2:47,"Check if there's any planned outage going on, ..."
2,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2798416,3,6/19/23 2:45,Closed,Host uscal-pixoslic1.lbj.is.smartgurucool.com ...,smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",6/21/23 2:47,"Check if there's any planned outage going on, ..."
3,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2798415,3,6/19/23 2:44,Closed,Host lbjvprox1.lbj.is.smartgurucool.com is Down,smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",6/21/23 2:46,"Check if there's any planned outage going on, ..."
4,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2798413,3,6/19/23 2:43,Closed,Host us-lbj-dc02.ad.smartgurucool.com is Down,smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",6/21/23 2:45,"Check if there's any planned outage going on, ..."


In [3]:
dataset.shape

(200000, 13)

In [4]:
dataset.tail()

,Type,Owned By Team,Customer Name,ID,Priority,Created Date Time,Status,Short Description,Category,Service,Subcategory,SLA Resolve By Deadline,Action Taken
199995,Incident,CSM_TEAM_CS_TCS_UX_Team,Solarwinds Customer,2661692,4,3/6/23 4:28,Closed,Host llbjartdist01.lbj.is.smartgurucool.com i...,smartgurucool - Unix and Linux,Cloud and Server Computing,"OS, HW, Performance Issue",3/9/23 4:31,"Check if there's any planned outage going on, ..."
199996,Incident,CSM_TEAM_CS_TCS_UX_Team,Solarwinds Customer,2661690,4,3/6/23 4:27,Closed,Host hacsvn02.hch.is.smartgurucool.com is Down,smartgurucool - Unix and Linux,Cloud and Server Computing,"OS, HW, Performance Issue",3/9/23 4:30,"Check if there's any planned outage going on, ..."
199997,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2661675,3,3/6/23 4:09,Closed,"On ksmdev1.cos.is.smartgurucool.com , CPU Util...",smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",3/8/23 4:11,Add 4 additional CPU cores to the server.
199998,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2661674,3,3/6/23 4:08,Closed,"On ksmwebrn2.bbn.is.smartgurucool.com , CPU Ut...",smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",3/8/23 4:10,Raise a Change Request (in case of CR required...
199999,Incident,CSM_TEAM_CS_TCS_Windows_VMWare_Team,Solarwinds Customer,2661672,3,3/6/23 4:07,Closed,"On my-png-dc02.ad.smartgurucool.com , CPU Util...",smartgurucool - Windows,Cloud and Server Computing,"OS, HW, Performance Issue",3/8/23 4:09,Raise a Change Request (in case of CR required...


### Data Cleaning

As we can see, there are lot of columns which are not required. So, we will select only those columns which are necessary for our project.

In [5]:
data = dataset[['Short Description','Action Taken']]
data.head(20)

,Short Description,Action Taken
0,Host lbjesx02.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
1,Host lbjavscan1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
2,Host uscal-pixoslic1.lbj.is.smartgurucool.com ...,"Check if there's any planned outage going on, ..."
3,Host lbjvprox1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
4,Host us-lbj-dc02.ad.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
5,Host us-lbj-dc01.ad.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
6,Host setoolslbj.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
7,Host pixoslic1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
8,Host pixoslic1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
9,Host wlbjcmdp01.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."


In [6]:
data.tail()

,Short Description,Action Taken
199995,Host llbjartdist01.lbj.is.smartgurucool.com i...,"Check if there's any planned outage going on, ..."
199996,Host hacsvn02.hch.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ..."
199997,"On ksmdev1.cos.is.smartgurucool.com , CPU Util...",Add 4 additional CPU cores to the server.
199998,"On ksmwebrn2.bbn.is.smartgurucool.com , CPU Ut...",Raise a Change Request (in case of CR required...
199999,"On my-png-dc02.ad.smartgurucool.com , CPU Util...",Raise a Change Request (in case of CR required...


### Label Encoding

In [7]:
# creating a label encoder object

label_encoder = LabelEncoder()
data['Action Taken Encoded'] = label_encoder.fit_transform(data['Action Taken'])
data.to_csv('dataset_LabelEncoder.csv', header=True, index=False)
data.head(20)

,Short Description,Action Taken,Action Taken Encoded
0,Host lbjesx02.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
1,Host lbjavscan1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
2,Host uscal-pixoslic1.lbj.is.smartgurucool.com ...,"Check if there's any planned outage going on, ...",4
3,Host lbjvprox1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
4,Host us-lbj-dc02.ad.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
5,Host us-lbj-dc01.ad.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
6,Host setoolslbj.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
7,Host pixoslic1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
8,Host pixoslic1.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4
9,Host wlbjcmdp01.lbj.is.smartgurucool.com is Down,"Check if there's any planned outage going on, ...",4


### Naive Bayes Classifier

In [8]:
# Assuming 'X' is your 'Short Description' column and 'y' is your 'Action Taken Encoded' column
X = data['Short Description']
y = data['Action Taken Encoded']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train

153248    For node wcosdsql1k.cos.is.smartgurucool.com, ...
67802     Warning For node spfirep1.cos.is.smartgurucool...
148889    On srsbkp3.srs.is.smartgurucool.com , CPU Util...
103093      Host wscscve1.scs.is.smartgurucool.com  is Down
104681     Host wdfwldp3k.dfw.is.smartgurucool.com  is Down
                                ...                        
119879    For pwsg.srs.is.smartgurucool.com MEMORY Utili...
103694    Host cosgrid192.cos.is.smartgurucool.com  is Down
131932    For ldfwlogs01.dfw.is.smartgurucool.com MEMORY...
146867     Host wkobcmdp1.kob.is.smartgurucool.com  is Down
121958    For node wcosopelecd1.cos.is.smartgurucool.com...
Name: Short Description, Length: 160000, dtype: object

In [10]:
y_train

153248    15
67802     15
148889    14
103093     4
104681     4
          ..
119879    12
103694     4
131932    10
146867     4
121958    15
Name: Action Taken Encoded, Length: 160000, dtype: int64

In [11]:
start = time.time()
# Initialize a CountVectorizer to convert text into numerical features
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_vectorized = vectorizer.transform(X_test)

# Initialize a Naive Bayes classifier
classifier = MultinomialNB()

# Train the classifier
classifier.fit(X_train_vectorized, y_train)

# Predict using the trained model
y_pred = classifier.predict(X_test_vectorized)
end = time.time()
print('Time Taken:',round((end-start)/60,4),'minutes')

Time Taken: 0.026 minutes


In [12]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {round(accuracy*100,2)}%")

Accuracy: 99.9%


### Support Vector Machine

In [13]:
start = time.time()
# Initialize a Support Vector Machine classifier
svm_classifier = SVC()

# Train the SVM classifier
svm_classifier.fit(X_train_vectorized, y_train)

# Predict using the trained SVM model
svm_y_pred = svm_classifier.predict(X_test_vectorized)
end = time.time()
print('Time Taken:',round((end-start)/60,4),'minutes')

Time Taken: 0.1795 minutes


In [14]:
# Evaluate the SVM model
svm_accuracy = accuracy_score(y_test, svm_y_pred)
print(f"SVM Accuracy: {round(svm_accuracy*100,2)}%")

SVM Accuracy: 99.99%


### Decision Tree Classifier

In [15]:
start = time.time()
# Initialize a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier()

# Train the Decision Tree classifier
decision_tree_classifier.fit(X_train_vectorized, y_train)

# Predict using the trained Decision Tree model
dt_y_pred = decision_tree_classifier.predict(X_test_vectorized)
end = time.time()
print('Time Taken:',round((end-start)/60,4),'minutes')

Time Taken: 0.0021 minutes


In [16]:
# Evaluate the Decision Tree model
dt_accuracy = accuracy_score(y_test, dt_y_pred)
print(f"Decision Tree Accuracy: {round(dt_accuracy*100,2)}%")

Decision Tree Accuracy: 100.0%


### Random Forest Classifier

In [17]:
start = time.time()
# Initialize a Random Forest classifier
random_forest_classifier = RandomForestClassifier()

# Train the Random Forest classifier
random_forest_classifier.fit(X_train_vectorized, y_train)

# Predict using the trained Random Forest model
rf_y_pred = random_forest_classifier.predict(X_test_vectorized)
end = time.time()
print('Time Taken:',round((end-start)/60,4),'minutes')

Time Taken: 0.1288 minutes


In [18]:
# Evaluate the Random Forest model
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"Random Forest Accuracy: {round(rf_accuracy*100,2)}%")

Random Forest Accuracy: 100.0%


### Gradient Boosting Classifier

In [26]:
start = time.time()
# Initialize a Gradient Boosting classifier
gradient_boosting_classifier = GradientBoostingClassifier()

# Train the Gradient Boosting classifier
gradient_boosting_classifier.fit(X_train_vectorized, y_train)

# Predict using the trained Gradient Boosting model
gb_y_pred = gradient_boosting_classifier.predict(X_test_vectorized)
end = time.time()
print('Time Taken:',round((end-start)/60,4),'minutes')

Time Taken: 2.599 minutes


In [20]:
# Evaluate the Gradient Boosting model
gb_accuracy = accuracy_score(y_test, gb_y_pred)
print(f"Gradient Boosting Accuracy: {round(gb_accuracy*100,2)}%")

Gradient Boosting Accuracy: 100.0%


In [21]:
#Observation: This model takes around 3 minutes of time to train.

### Application Backend Function

In [22]:
# As we are getting 100% accuracy level and minimum time-execution with Decision Tree Classifier, so we are choosing that for building our app
data = pd.read_csv('dataset_LabelEncoder.csv')
model = DecisionTreeClassifier()

# as 'Short Description' is in text, so we need to convert it into text features using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Short Description'])
y = data['Action Taken Encoded']

# model fitting
model.fit(X,y)

DecisionTreeClassifier()

In [23]:
# reverse mapping for label encoder
reverse_mapping = {}
for i in data.index:
    if int(data['Action Taken Encoded'][i]) not in reverse_mapping:
        reverse_mapping[int(data['Action Taken Encoded'][i])] = str(data['Action Taken'][i])

for k,v in reverse_mapping.items():
    print(k,'-->',v)

4 --> Check if there's any planned outage going on, otherwise go to the vCenter and power on the server. Also, find out the RCA and send it to the end users.
14 --> Raise a Change Request (in case of CR required server) and add 4 additional cores of CPU as server might get hung. Already the utilization is very high.
15 --> This alert got triggered mistakenly. No memory percentage is mentioned. Kindly send a mail to Tools Team to take necessary action at the earliest.
1 --> Add 4 additional CPU cores to the server.
7 --> Inform the user to do the necessary housekeeping, but still no much scope then increase the drive by 10 GB.
5 --> First, convince the server owner that space addition in C:\ is not recommended and suggest to move the non-OS items to the non-OS drive like E:\ or D:\ etc. If still server owner says that housekepping already done and no further housekeeping is pending, in that case, get the approval from SMEs and increase the C:\ by additional 5 GB.
12 --> Need to add 2 GB

In [24]:
# main logic
def predict_action_taken(short_description):
    # preprocessing user input
    input_vector = vectorizer.transform([short_description])
    
    # make prediction
    predicted_encoded_action = model.predict(input_vector)[0]
    
    predicted_action = reverse_mapping.get(predicted_encoded_action,'unknown')
    
    return predicted_action

user_input = 'grapes.ggn.is.smartgurucool.com C:\ Label:OS Disk  Serial Number 884ECD66 8 %'
predicted_action = predict_action_taken(user_input)
print(f"Predicted Action Taken: {predicted_action}")

Predicted Action Taken: First, convince the server owner that space addition in C:\ is not recommended and suggest to move the non-OS items to the non-OS drive like E:\ or D:\ etc. If still server owner says that housekepping already done and no further housekeeping is pending, in that case, get the approval from SMEs and increase the C:\ by additional 5 GB.
